# N2 Exercise
Before starting the exercise, we will verify that you have access to the cluster, can log in, and can run a calculation using the remotemanager.

In [ ]:
! pip install remotemanager > /dev/null 2> /dev/null
! pip install requests > /dev/null 2> /dev/null
! pip install PyBigDFT > /dev/null 2> /dev/null

In [ ]:
from remotemanager.connection.computers.base import BaseComputer, \
    required, optional


def saga_parser(resources):
    from remotemanager.connection.computers.base import format_time
    output = []
    
    cores_per_node = 32
    if resources['mpi_per_node']:
        if not resources['nodes']:
            raise RuntimeError('nodes must be specified with mpi_per_node')
        if resources['mpi_per_node'].value > cores_per_node:
            raise ValueError(f'mpi_per_node cannot be greater than cores_per_node ({cores_per_node})')

        nodes = resources['nodes'].value
        ntasks = resources['mpi_per_node'].value * nodes
        omp = resources["omp"].value
        
        output.append(f"#SBATCH --nodes={nodes}")
        output.append(f"#SBATCH --ntasks={ntasks}")
    else:
        # extract our mpi and omp request, calculate total cores
        mpi = resources["mpi"].value
        omp = resources["omp"].value
        ntasks = mpi * omp
        print(f'{ntasks} total mpi requested')
        
        nodes = int(round(max(1, ntasks / cores_per_node)))
        output.append(f"#SBATCH --nodes={nodes}")
        output.append(f"#SBATCH --ntasks={ntasks}")

    if not resources["partition"]:
        resources["partition"] = resources["queue"].value

    #options['--export'] = 'none'    
    for k, v in resources.items():
        if k in ['nodes', 'mpi' 'mpi_per_node']:
            continue
        if k == "time":
            output.append(f"#SBATCH --{v.flag}={format_time(v.value)}")
        elif v:
            output.append(f"#SBATCH --{v.flag}={v.value}")
    
    output.append(f'\nexport OMP_NUM_THREADS={omp}')

    return output


class saga(BaseComputer):
    """
    class for connecting to Saga, note settings correspond to the large memory nodes which will be used for the tutorial
    """

    def __init__(self,
                 passfile: str,
                 module_purge=False,
                 **kwargs):

        kwargs['passfile'] = passfile

        if 'host' not in kwargs:
            kwargs['host'] = 'fram.sigma2.no'

        super().__init__(**kwargs)

        self.submitter = 'sbatch'
        self.shebang = '#!/bin/bash'

        self.mpi = required('ntasks')
        self.mpi_per_node = required('tasks-per-node')
        self.omp = required('cpus-per-task')
        self.walltime = required('time')
        self.memory = required('mem-per-cpu')        
        self.nodes = optional('nodes')
        self.jobname = optional('job-name')
        self.outfile = optional('output')
        self.errfile = optional('error')
        self.account = optional('account')
        
        #self.queue = optional('')
        self.partition = optional('partition', 'bigmem')

        self.choice_groupings = [["mpi", "mpi_per_node"]]

        self._parser = saga_parser

In [ ]:
from computers.ims import 
url = saga(user=YOURID, passfile=YOURPASSFILE)
url.mpi = 1
url.omp = 1
url.time = "00:10:00"
url.account = YOURID
url.extra = """
export BIGDFT_MPIRUN='srun --distribution=cyclic:cyclic'
module load intel/2022b
source /cluster/home/lra040/PROJECT_WORK/bigdft-suite/Build/install/bin/bigdftvars.sh
"""

In [ ]:
connection.cmd('ls')

In [ ]:
%load_ext remotemanager

In [ ]:
%%sanzu url=url
return 1

In [ ]:
print(magic_dataset.results[0])

### Exercise

Compare the values of the HOMO and HOMO-1 eigenvalues for the LDA and the HF run.
Change the values of the *hgrid* and *crmult* to find the converged values.

Note that, both in the LDA and in the HF calculation, a norm-conserving PSP is used.

The results can be compared to all-electron calculations, done with different basis sets, from references (units are eV) 
(1) S.&nbsp;Hamel <i>et&nbsp;al.</i> J. Electron Spectrospcopy and Related Phenomena 123 (2002) 345-363 and (2) P.&nbsp;Politzer, F.&nbsp;Abu-Awwad, Theor. Chem. Acc. (1998), 99, 83-87:

eigenvalues          | LDA(1) | HF(1) | HF(2) | (Exp.)
---------------------|--------|-------|-------|-------
3&sigma;<sub>g</sub> | 10.36  | 17.25 | 17.31 | (15.60)
1&pi;<sub>u</sub>    | 11.84  | 16.71 | 17.02 | (16.98)
2&sigma;<sub>u</sub> | 13.41  | 21.25 | 21.08 | (18.78)

The results depends, of course, on the precision chosen for the calculation, and of the presence of the pseudopotential.
As it is well-known, the pseudopotential appoximation is however much less severe than the approximation induced by typical XC functionals. We might see that, even in the HF case, the presence of a LDA-based pseudopotential (of rather good quality) does not alter so much the results. 

Here you can find the values from BigDFT calculation using a very good precision (*hgrid=0.3*, *crmult=7.0*). 
Note that 1 Ha=27.21138386 eV.
 
eigenvalues          | LDA   | HF
---------------------|-------|------
3&sigma;<sub>g</sub> | 10.40 | 17.32
1&pi;<sub>u</sub>    | 11.75 | 16.62
2&sigma;<sub>u</sub> | 13.52 | 21.30

How much do these values differ from the calculation with default parameters? Do they converge to a given value?
What is the *correlation* for the N2 molecule in (PSP) LDA?

See here the [solution](https://github.com/BigDFT-group/bigdft-school/blob/main/CCP_tutorials/1.H.N2Exercise.ipynb).